# Install / Import / Load libraries

In [ ]:
!pip install tensorflow==1.15

In [ ]:
# !pip install --quiet lucid==0.0.5
!pip install --quiet lucid==0.3
!npm install -g svelte-cli@2.2.0

import numpy as np
import tensorflow as tf

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url, _display_html
import lucid.scratch.web.svelte as lucid_svelte

     |████████████████████████████████| 51kB 2.8MB/s 
/tools/node/bin/svelte -> /tools/node/lib/node_modules/svelte-cli/bin.js
+ svelte-cli@2.2.0
added 2 packages from 1 contributor in 2.081s


In [ ]:
import pkg_resources
pkg_resources.get_distribution("lucid").version

'0.3.8'

In [ ]:
model1 = models.InceptionV1()
model2=models.VGG19_caffe()
model1.load_graphdef()
model2.load_graphdef()

W0501 10:02:39.183881 140576163706752 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/lucid/misc/io/writing.py:62: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0501 10:02:39.185848 140576163706752 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/lucid/misc/io/writing.py:70: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

W0501 10:02:40.966193 140576163706752 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/lucid/modelzoo/util.py:38: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



In [ ]:
models.VGG19_caffe.layers

[{'name': 'conv1_1/conv1_1', 'size': 64, 'type': 'conv'},
 {'name': 'conv1_2/conv1_2', 'size': 64, 'type': 'conv'},
 {'name': 'conv2_1/conv2_1', 'size': 128, 'type': 'conv'},
 {'name': 'conv2_2/conv2_2', 'size': 128, 'type': 'conv'},
 {'name': 'conv3_1/conv3_1', 'size': 256, 'type': 'conv'},
 {'name': 'conv3_2/conv3_2', 'size': 256, 'type': 'conv'},
 {'name': 'conv3_3/conv3_3', 'size': 256, 'type': 'conv'},
 {'name': 'conv3_4/conv3_4', 'size': 256, 'type': 'conv'},
 {'name': 'conv4_1/conv4_1', 'size': 512, 'type': 'conv'},
 {'name': 'conv4_2/conv4_2', 'size': 512, 'type': 'conv'},
 {'name': 'conv4_3/conv4_3', 'size': 512, 'type': 'conv'},
 {'name': 'conv4_4/conv4_4', 'size': 512, 'type': 'conv'},
 {'name': 'conv5_1/conv5_1', 'size': 512, 'type': 'conv'},
 {'name': 'conv5_2/conv5_2', 'size': 512, 'type': 'conv'},
 {'name': 'conv5_3/conv5_3', 'size': 512, 'type': 'conv'},
 {'name': 'conv5_4/conv5_4', 'size': 512, 'type': 'conv'},
 {'name': 'fc6/fc6', 'size': 4096, 'type': 'dense'},
 {'na

In [ ]:
models.InceptionV1.layers

[{'name': 'conv2d0', 'size': 64, 'type': 'conv'},
 {'name': 'conv2d1', 'size': 64, 'type': 'conv'},
 {'name': 'conv2d2', 'size': 192, 'type': 'conv'},
 {'name': 'mixed3a', 'size': 256, 'type': 'conv'},
 {'name': 'mixed3b', 'size': 480, 'type': 'conv'},
 {'name': 'mixed4a', 'size': 508, 'type': 'conv'},
 {'name': 'mixed4b', 'size': 512, 'type': 'conv'},
 {'name': 'mixed4c', 'size': 512, 'type': 'conv'},
 {'name': 'mixed4d', 'size': 528, 'type': 'conv'},
 {'name': 'mixed4e', 'size': 832, 'type': 'conv'},
 {'name': 'mixed5a', 'size': 832, 'type': 'conv'},
 {'name': 'mixed5b', 'size': 1024, 'type': 'conv'},
 {'name': 'head0_bottleneck', 'size': 128, 'type': 'conv'},
 {'name': 'nn0', 'size': 1024, 'type': 'dense'},
 {'name': 'softmax0', 'size': 1008, 'type': 'dense'},
 {'name': 'head1_bottleneck', 'size': 128, 'type': 'conv'},
 {'name': 'nn1', 'size': 1024, 'type': 'dense'},
 {'name': 'softmax1', 'size': 1008, 'type': 'dense'},
 {'name': 'softmax2', 'size': 1008, 'type': 'dense'}]

**Channel Attribution Widget**


In [ ]:
#taken from https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/building-blocks/AttrChannel.ipynb

%%html_define_svelte ChannelAttrWidget

<div class="figure">
  <div class="channel_list" >
    {{#each attrsPos as attr}}
    <div class="entry">
      <div class="sprite" style="background-image: url({{spritemap_url}}); width: {{sprite_size}}px; height: {{sprite_size}}px; background-position: -{{sprite_size*(attr.n%sprite_n_wrap)}}px -{{sprite_size*Math.floor(attr.n/sprite_n_wrap)}}px;"></div>
      <div class="value" style="background-color: hsl({{(attr.v > 0)? 210 : 0}}, {{100*Math.abs(attr.v)/1.8}}%, {{100-30*Math.abs(attr.v)/1.8}}%)">{{attr.v}}</div>
    </div>
    {{/each}}
    {{#if attrsPos.length > 5}}
    <br style="clear:both;">
    <br style="clear:both;">
    {{/if}}
    <div class="gap">...</div>
    {{#each attrsNeg as attr}}
    <div class="entry">
      <div class="sprite" style="background-image: url({{spritemap_url}}); width: {{sprite_size}}px; height: {{sprite_size}}px; background-position: -{{sprite_size*(attr.n%sprite_n_wrap)}}px -{{sprite_size*Math.floor(attr.n/sprite_n_wrap)}}px;"></div>
      <div class="value" style="background-color: hsl({{(attr.v > 0)? 210 : 0}}, {{100*Math.abs(attr.v)/1.8}}%, {{100-30*Math.abs(attr.v)/1.8}}%)">{{attr.v}}</div>
    </div>
    {{/each}}
  </div>
  <br style="clear:both">
</div>


<style>
  .entry{
    float: left;
    margin-right: 4px;
  }
  .gap {
    float: left;
    margin: 8px;
    font-size: 400%;
  }
</style>

<script>
    
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        spritemap_url: "",
        sprite_size: 110,
        sprite_n_wrap: 22,
        attrsPos: [],
        attrsNeg: [],
      };
    },
    computed: {
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_YewYal/ChannelAttrWidget_ee1ec433_9505_48d9_a47f_75874ce7ca27.html > /tmp/svelte_YewYal/ChannelAttrWidget_ee1ec433_9505_48d9_a47f_75874ce7ca27.js
svelte version 1.64.1
compiling ../tmp/svelte_YewYal/ChannelAttrWidget_ee1ec433_9505_48d9_a47f_75874ce7ca27.html...



**Bars Widget**



In [ ]:
#taken from https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/building-blocks/AttrChannel.ipynb
%%html_define_svelte BarsWidget

<div class="figure">
  <div class="channel_list" >
    {{#each vals as val}}
    <div class="bar" style="height: {{15*Math.abs(val)}}px; background-color: hsl({{(val > 0)? 210 : 0}}, {{Math.max(90, 110*Math.abs(val)/1.8)}}%, {{Math.min(80, 100-40*Math.abs(val)/1.8)}}%);">
    </div>
    {{/each}}
  </div>
  <br style="clear:both">
</div>


<style>
  .channel_list {
    background-color: #FEFEFE;
  }
  .bar {
    width: 1.5px;
    height: 10px;
    display: inline-block;
  }
</style>

<script>
  
  export default {
    data () {
      return {
        vals: []
      };
    }
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_YewYal/BarsWidget_0043bfd5_c5a1_4fc4_b1fd_85aeb93a1818.html > /tmp/svelte_YewYal/BarsWidget_0043bfd5_c5a1_4fc4_b1fd_85aeb93a1818.js
svelte version 1.64.1
compiling ../tmp/svelte_YewYal/BarsWidget_0043bfd5_c5a1_4fc4_b1fd_85aeb93a1818.html...



# Loading Spritemaps

In [ ]:
layer_spritemap_sizes_incv1 = {
    'mixed3a' : 16,
    'mixed3b' : 21,
    'mixed4a' : 22,
    'mixed4b' : 22,
    'mixed4c' : 22,
    'mixed4d' : 22,
    'mixed4e' : 28,
    'mixed5a' : 28,
  }
  
layer_spritemap_sizes_vgg19 = {
    'conv4_4/conv4_4' : 10,
    'conv5_3/conv5_3':10
  }
def googlenet_spritemap_incv1(layer):
  assert layer in layer_spritemap_sizes_incv1
  size = layer_spritemap_sizes_incv1[layer]
  url = "https://storage.googleapis.com/lucid-static/building-blocks/googlenet_spritemaps/sprite_%s_channel_alpha.jpeg" % layer
  return size, url

def googlenet_spritemap_vgg19(layer):
  assert layer in layer_spritemap_sizes_vgg19
  size = layer_spritemap_sizes_vgg19[layer]
  #"https://i.ibb.co/vZgwzvD/spritesheet2021-04-30-T17-43-41.png" - url for conv4_4/conv4_4
  url = 'https://i.ibb.co/MsYTGg4/spritesheet2021-04-30-T23-57-36.png'
  return size, url


**Attribution Code for Inception V1**

In [ ]:
#taken from https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/building-blocks/AttrChannel.ipynb
def score_f_incv1(logit, name):
  if name is None:
    return 0
  elif name == "logsumexp":
    base = tf.reduce_max(logit)
    return base + tf.log(tf.reduce_sum(tf.exp(logit-base)))
  elif name in model1.labels:
    return logit[model1.labels.index(name)]
  else:
    raise RuntimeError("Unsupported")

def channel_attr_incv1(img, layer, class1, class2, n_show=4):

  # Set up a graph for doing attribution
  with tf.Graph().as_default(), tf.Session() as sess:
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model1, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    # Compute gradient
    logit = T("softmax2_pre_activation")[0]
    score = score_f_incv1(logit, class1) - score_f_incv1(logit, class2)
    t_grad = tf.gradients([score], [T(layer)])[0]
    grad = t_grad.eval()


    attr = (grad*acts)[0]
    
    # Reduce down to channels.
    channel_attr = attr.sum(0).sum(0)


  
  # Get spritemaps
  
  
  spritemap_n, spritemap_url = googlenet_spritemap_incv1(layer)
  
  # Let's show the distribution of attributions
  print "Distribution of attribution accross channels:"
  print ""
  lucid_svelte.BarsWidget({"vals" : [float(v) for v in np.sort(channel_attr)[::-1]]})

  # Let's pick the most extreme channels to show
  ns_pos = list(np.argsort(-channel_attr)[:n_show])
  ns_neg = list(np.argsort(channel_attr)[:n_show][::-1])
  
  
  print ""
  print "Top", n_show, "channels in each direction:"
  print ""
  lucid_svelte.ChannelAttrWidget({
    "spritemap_url": spritemap_url,
    "sprite_size": 110,
    "sprite_n_wrap": spritemap_n,
    "attrsPos": [{"n": n, "v": str(float(channel_attr[n]))[:5]} for n in ns_pos],
    "attrsNeg": [{"n": n, "v": str(float(channel_attr[n]))[:5]} for n in ns_neg] 
  })


**Attribution code for VGG-19**

In [ ]:

def score_f_vgg19(logit, name):
  if name is None:
    return 0
  elif name == "logsumexp":
    base = tf.reduce_max(logit)
    return base + tf.log(tf.reduce_sum(tf.exp(logit-base)))
  elif name in model2.labels:
    return logit[model2.labels.index(name)]
  else:
    raise RuntimeError("Unsupported")
def channel_attr_vgg19(img, layer, class1, class2, n_show=4):

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session() as sess:
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model2, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
   
    # Compute gradient
    logit = T("prob")[0]
    score = score_f_vgg19(logit, class1) - score_f_vgg19(logit, class2)
    t_grad = tf.gradients([score], [T(layer)])[0]
    grad = t_grad.eval()
    
   
    attr = (grad*acts)[0]
    
    # Then we reduce down to channels.
    channel_attr = attr.sum(0).sum(0)

  
  # Get spritemaps
  
  
  spritemap_n, spritemap_url = googlenet_spritemap_vgg19(layer)
  
  # Let's show the distribution of attributions
  print "Distribution of attribution accross channels:"
  print ""
  lucid_svelte.BarsWidget({"vals" : [float(v) for v in np.sort(channel_attr)[::-1]]})

  # Let's pick the most extreme channels to show
  ns_pos = list(np.argsort(-channel_attr)[:n_show])
  ns_neg = list(np.argsort(channel_attr)[:n_show][::-1])
  
  
  print ""
  print "Top", n_show, "channels in each direction:"
  print ""
  lucid_svelte.ChannelAttrWidget({
    "spritemap_url": spritemap_url,
    "sprite_size": 128,
    "sprite_n_wrap": spritemap_n,
    "attrsPos": [{"n": n, "v": str(float(channel_attr[n]))[:5]} for n in ns_pos],
    "attrsNeg": [{"n": n, "v": str(float(channel_attr[n]))[:5]} for n in ns_neg] 
  })


### Code for generating neuron visualizations which will be added to the spritemap for the layer conv5_3/conv5_3  of VGG-19

In [ ]:
# import time
# import cv2
# from google.colab import files
# st=time.time()
# for i in range(512):
#     a = render.render_vis(model, 'conv5_3/conv5_3:{}'.format(i))
#     cv2.imwrite('test{}.jpg'.format(i),a[0][0]*255)
#     files.download( "test{}.jpg".format(i) ) 
#     print(time.time()-st)

In [ ]:
# !zip -r /content/file.zip /content/

In [ ]:
# files.download("/content/file.zip")

# Channel attributions for Inception V1

In [ ]:
img = load("https://upload.wikimedia.org/wikipedia/commons/d/d0/German_Shepherd_-_DSC_0346_%2810096362833%29.jpg")
channel_attr_incv1(img, "mixed4d", "German shepherd", "Maltese dog", n_show=5)

Distribution of attribution accross channels:




Top 5 channels in each direction:



In [ ]:

img = load("https://247wallst.com/wp-content/uploads/2019/10/gettyimages-596088724.jpg")
channel_attr_incv1(img, "mixed4d", "cheeseburger", "hotdog", n_show=5)

Distribution of attribution accross channels:




Top 5 channels in each direction:



# Channel attributions for VGG-19

In [ ]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png")
channel_attr_vgg19(img, "conv5_3/conv5_3", "Labrador retriever", "tiger cat", n_show=5)

Distribution of attribution accross channels:




Top 5 channels in each direction:

